In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
import json
import os
from models.all_model import *
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from datetime import datetime
import numpy as np

Using TensorFlow backend.


In [2]:
network="alexnet" # can be one of options: alexnet,vgg,resnet,googlenet
pretrained_documents=True # pretrained from Document or not
pretrained_imagenet = False # pretrained from Imagenet or not
# Both pretrained_documents and pretrained_imagenet should not be True 
# at the same time
dataset="tobaco"

In [3]:
n_classes=10
num_EPOCHS = 10
batch_size = 16
if network == "alexnet":
    IMG_SIZE=227
else:
    IMG_SIZE = 224
def init_model():
    if network == "vgg":
        model = vgg(n_classes,IMG_SIZE,dataset,pretrained_imagenet)
        # in document pretrained mode, we just train few last FC layers
        if pretrained_documents:
            for layer in model.layers:
                layer.trainable = False
            for layer in model.layers[-3:]:
                layer.trainable = True
            # Load check point 
            model.load_weights("./checkpoint_RVL_CDIP_vgg_pretrained/weights-05-0.89.hdf5",by_name=True)
   
    elif network == "googlenet":   
        model = googlenet(n_classes,IMG_SIZE,dataset,pretrained_imagenet)
        # in document pretrained mode, we just train few last FC layers
        if pretrained_documents:
            for layer in model.layers:
                layer.trainable = False
            for layer in model.layers[-1:]:
                layer.trainable = True
            # Load check point 
            model.load_weights("./checkpoint_RVL_CDIP_googlenet_pretrained/weights-08-0.89.hdf5",by_name=True)
    elif network == "resnet":
        model = resnet(n_classes,IMG_SIZE,dataset,pretrained_imagenet)
        # in document pretrained mode, we just train few last FC layers
        if pretrained_documents: 
            for layer in model.layers:
                layer.trainable = False
            for layer in model.layers[-1:]:
                layer.trainable = True
            # Load check point 
            model.load_weights("./checkpoint_RVL_CDIP_resnet_pretrained/weights-04-0.89.hdf5",by_name=True)
    elif network == "alexnet":
        IMG_SIZE = 227
        model = Alexnet(n_classes,IMG_SIZE,dataset,pretrained_imagenet)
        # in document pretrained mode, we just train few last FC layers
        if pretrained_documents:
            for layer in model.layers:
                layer.trainable = False
            for layer in model.layers[-6:]:
                layer.trainable = True
            # Load check point 
            model.load_weights("./checkpoint_RVL_CDIP_alexnet_no_pretrained/weights-08-0.84.hdf5",by_name=True)
    return model


In [5]:
import numpy as np
import random
import cv2
image_group_by_label=[]
#Group all image dataset Tobacco by label
for index,label in enumerate(os.listdir("Tobacco3482")):
    all_image_files  =  os.listdir("Tobacco3482/"+label)
    all_images = []
    for file in all_image_files:
        if file.endswith("jpg"):
            img = cv2.imread("Tobacco3482/"+label+"/"+file)
            img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            img = img/255.0
            all_images.append(img)
    image_group_by_label.append(all_images)


In [6]:
plot = []
for num_example_train in range(20,110,20):
    mean_acc = 0.0
    for i in range(0,10): # randomly create 10 partions
        keras.backend.clear_session()
        model=init_model()
        model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
        X_train = []
        Y_train_index = []
        X_val = []
        Y_val_index = []
        X_test  = []
        Y_test_index  = []
        for index,group in enumerate(image_group_by_label):
            random.shuffle(group)
            #choose 80% number of example per class to train
            image_train = group[0:int(num_example_train/10*8)]
            Y_train_index += [index]*len(image_train)
            X_train = X_train + image_train
              #choose 20% number of example per class to val
            image_val = group[int(num_example_train/10*8):num_example_train]
            Y_val_index+= [index]*len(image_val) 
            X_val = X_val +image_val
            # the rest are for testing
            image_test = group[num_example_train::]
            Y_test_index += [index]*len(image_test)  
            X_test = X_test + image_test
            
        X_train=np.array(X_train, dtype=np.float32)
        Y_train_index=np.array(Y_train_index,dtype=np.int64)
        
        X_test = np.array(X_test,dtype=np.float32)
        Y_test_index = np.array(Y_test_index,dtype=np.int64)
        
        X_val = np.array(X_val,dtype=np.float32)
        Y_val_index = np.array(Y_val_index,dtype=np.int64)
        
        randomize = np.arange(len(X_train))
        np.random.shuffle(randomize)
        X_train = X_train[randomize]
        Y_train_index = Y_train_index[randomize]
        Y_train =  np.eye(n_classes)[Y_train_index]
        Y_val = np.eye(n_classes)[Y_val_index]
        Y_test = np.eye(n_classes)[Y_test_index]
        print(len(X_val))
        model.fit(X_train,Y_train,epochs=num_EPOCHS, batch_size=batch_size,validation_data=(X_val,Y_val))
        loss,acc = model.evaluate(X_test,Y_test,batch_size=batch_size)
        mean_acc += acc 
    plot.append( [mean_acc/10.0,num_example_train] ) #report the median accuracy

40
Instructions for updating:
Use tf.cast instead.
Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 4s 28ms/step - loss: 1.8605 - acc: 0.5625 - val_loss: 1.2176 - val_acc: 0.8000
Epoch 2/10
160/160 [==============================] - 0s 3ms/step - loss: 0.5691 - acc: 0.9625 - val_loss: 1.4324 - val_acc: 0.7000
Epoch 3/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4781 - acc: 0.9938 - val_loss: 1.8178 - val_acc: 0.6250
Epoch 4/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4601 - acc: 1.0000 - val_loss: 1.7513 - val_acc: 0.7000
Epoch 5/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4547 - acc: 1.0000 - val_loss: 1.6746 - val_acc: 0.7500
Epoch 6/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4503 - acc: 1.0000 - val_loss: 1.6717 - val_acc: 0.7250
Epoch 7/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4456 - acc: 1.0000 - val_loss: 1.66

160/160 [==============================] - 0s 3ms/step - loss: 0.4501 - acc: 1.0000 - val_loss: 1.8091 - val_acc: 0.6750
Epoch 7/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4457 - acc: 1.0000 - val_loss: 1.6948 - val_acc: 0.6750
Epoch 8/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4411 - acc: 1.0000 - val_loss: 1.6566 - val_acc: 0.6750
Epoch 9/10
160/160 [==============================] - 0s 3ms/step - loss: 0.4367 - acc: 1.0000 - val_loss: 1.6582 - val_acc: 0.6750
Epoch 10/10
3282/3282 [==============================] - 3s 1ms/step
40
Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 4s 23ms/step - loss: 1.8129 - acc: 0.5687 - val_loss: 1.4573 - val_acc: 0.6000
Epoch 2/10
160/160 [==============================] - 0s 3ms/step - loss: 0.6327 - acc: 0.9625 - val_loss: 1.2627 - val_acc: 0.7500
Epoch 3/10
160/160 [==============================] - 0s 3ms/step - loss: 0.5014 - acc: 0.9875 - val_

320/320 [==============================] - 1s 2ms/step - loss: 0.4874 - acc: 0.9969 - val_loss: 1.1481 - val_acc: 0.7875
Epoch 4/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4590 - acc: 1.0000 - val_loss: 1.3435 - val_acc: 0.7625
Epoch 5/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4514 - acc: 1.0000 - val_loss: 1.1127 - val_acc: 0.8125
Epoch 6/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4437 - acc: 1.0000 - val_loss: 1.1061 - val_acc: 0.8250
Epoch 7/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4376 - acc: 1.0000 - val_loss: 1.0686 - val_acc: 0.8500
Epoch 8/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4317 - acc: 1.0000 - val_loss: 1.0791 - val_acc: 0.8500
Epoch 9/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4259 - acc: 1.0000 - val_loss: 1.0753 - val_acc: 0.8375
Epoch 10/10
3082/3082 [==============================] - 3s 932us/step
80
Train on 320 

3082/3082 [==============================] - 3s 991us/step
80
Train on 320 samples, validate on 80 samples
Epoch 1/10
320/320 [==============================] - 4s 13ms/step - loss: 1.5358 - acc: 0.6687 - val_loss: 1.1916 - val_acc: 0.7750
Epoch 2/10
320/320 [==============================] - 1s 3ms/step - loss: 0.5926 - acc: 0.9563 - val_loss: 1.5035 - val_acc: 0.7250
Epoch 3/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4821 - acc: 0.9937 - val_loss: 1.3669 - val_acc: 0.7000
Epoch 4/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4538 - acc: 1.0000 - val_loss: 1.2484 - val_acc: 0.7625
Epoch 5/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4458 - acc: 1.0000 - val_loss: 1.2905 - val_acc: 0.7375
Epoch 6/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4390 - acc: 1.0000 - val_loss: 1.2927 - val_acc: 0.7125
Epoch 7/10
320/320 [==============================] - 1s 3ms/step - loss: 0.4324 - acc: 1.0000 - val

480/480 [==============================] - 1s 3ms/step - loss: 0.4195 - acc: 1.0000 - val_loss: 1.3318 - val_acc: 0.7750
Epoch 9/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4114 - acc: 1.0000 - val_loss: 1.3815 - val_acc: 0.7917
Epoch 10/10
2882/2882 [==============================] - 3s 898us/step
120
Train on 480 samples, validate on 120 samples
Epoch 1/10
480/480 [==============================] - 5s 10ms/step - loss: 1.4669 - acc: 0.6812 - val_loss: 1.2238 - val_acc: 0.7583
Epoch 2/10
480/480 [==============================] - 1s 3ms/step - loss: 0.5852 - acc: 0.9729 - val_loss: 1.2965 - val_acc: 0.7417
Epoch 3/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4786 - acc: 0.9896 - val_loss: 1.2240 - val_acc: 0.7917
Epoch 4/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4761 - acc: 0.9958 - val_loss: 1.3078 - val_acc: 0.7750
Epoch 5/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4516 - acc: 0.9958 - 

480/480 [==============================] - 1s 3ms/step - loss: 0.4377 - acc: 1.0000 - val_loss: 1.0922 - val_acc: 0.7917
Epoch 6/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4290 - acc: 1.0000 - val_loss: 1.2049 - val_acc: 0.7583
Epoch 7/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4199 - acc: 1.0000 - val_loss: 1.1641 - val_acc: 0.7583
Epoch 8/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4113 - acc: 1.0000 - val_loss: 1.1674 - val_acc: 0.7583
Epoch 9/10
480/480 [==============================] - 1s 3ms/step - loss: 0.4030 - acc: 1.0000 - val_loss: 1.1475 - val_acc: 0.7917
Epoch 10/10
2882/2882 [==============================] - 3s 1ms/step
120
Train on 480 samples, validate on 120 samples
Epoch 1/10
480/480 [==============================] - 4s 9ms/step - loss: 1.4512 - acc: 0.6771 - val_loss: 1.4695 - val_acc: 0.7000
Epoch 2/10
480/480 [==============================] - 1s 3ms/step - loss: 0.6017 - acc: 0.9458 - val

640/640 [==============================] - 2s 3ms/step - loss: 0.5879 - acc: 0.9625 - val_loss: 1.0633 - val_acc: 0.7875
Epoch 3/10
640/640 [==============================] - 2s 3ms/step - loss: 0.5039 - acc: 0.9813 - val_loss: 1.1810 - val_acc: 0.7688
Epoch 4/10
640/640 [==============================] - 2s 3ms/step - loss: 0.4676 - acc: 0.9937 - val_loss: 1.2638 - val_acc: 0.7812
Epoch 5/10
640/640 [==============================] - 2s 3ms/step - loss: 0.4584 - acc: 0.9937 - val_loss: 1.2496 - val_acc: 0.7625
Epoch 6/10
640/640 [==============================] - 2s 3ms/step - loss: 0.4439 - acc: 0.9922 - val_loss: 1.2779 - val_acc: 0.7562
Epoch 7/10
640/640 [==============================] - 2s 3ms/step - loss: 0.4257 - acc: 0.9984 - val_loss: 1.4157 - val_acc: 0.7375
Epoch 8/10
640/640 [==============================] - 2s 3ms/step - loss: 0.4278 - acc: 0.9953 - val_loss: 1.1839 - val_acc: 0.7562
Epoch 9/10
640/640 [==============================] - 2s 3ms/step - loss: 0.5199 - acc:

Epoch 10/10
2682/2682 [==============================] - 3s 945us/step
200
Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 5s 7ms/step - loss: 1.3292 - acc: 0.7275 - val_loss: 1.1554 - val_acc: 0.7750
Epoch 2/10
800/800 [==============================] - 2s 3ms/step - loss: 0.6077 - acc: 0.9500 - val_loss: 1.5274 - val_acc: 0.7500
Epoch 3/10
800/800 [==============================] - 2s 3ms/step - loss: 0.5177 - acc: 0.9800 - val_loss: 1.1875 - val_acc: 0.8200
Epoch 4/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4683 - acc: 0.9912 - val_loss: 1.2152 - val_acc: 0.8000
Epoch 5/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4565 - acc: 0.9888 - val_loss: 1.2054 - val_acc: 0.8150
Epoch 6/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4386 - acc: 0.9975 - val_loss: 1.2880 - val_acc: 0.8200
Epoch 7/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4342 - acc:

800/800 [==============================] - 2s 3ms/step - loss: 0.4177 - acc: 1.0000 - val_loss: 1.3462 - val_acc: 0.7850
Epoch 8/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4062 - acc: 1.0000 - val_loss: 1.4270 - val_acc: 0.7700
Epoch 9/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4110 - acc: 0.9975 - val_loss: 1.5508 - val_acc: 0.7600
Epoch 10/10
2482/2482 [==============================] - 2s 920us/step
200
Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 5s 7ms/step - loss: 1.3195 - acc: 0.7438 - val_loss: 1.2782 - val_acc: 0.7250
Epoch 2/10
800/800 [==============================] - 2s 3ms/step - loss: 0.6290 - acc: 0.9450 - val_loss: 1.4168 - val_acc: 0.7100
Epoch 3/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4981 - acc: 0.9813 - val_loss: 1.6971 - val_acc: 0.7150
Epoch 4/10
800/800 [==============================] - 2s 3ms/step - loss: 0.4617 - acc: 0.9925 - v

In [7]:
plot=np.array(plot,dtype=np.float32)
mode =""

if pretrained_documents:
    mode="_Document_pretrained"
elif pretrained_imagenet:
    mode="_Imagenet_pretrained"
else:
    mode="_no_pretrained"
    
file_name="./plot_info/"+network+"_"+dataset+mode+".npy"
if pretrained_documents: #we just only save_weight of pretrained document
    model.save_weights("./tobaco_checkpoint/"+network+"_"+dataset+"_Document_pretrained"+".hdf5")
np.save(file_name,plot)

In [8]:
plot

array([[  0.71867764,  20.        ],
       [  0.7590526 ,  40.        ],
       [  0.777238  ,  60.        ],
       [  0.7634974 ,  80.        ],
       [  0.7648268 , 100.        ]], dtype=float32)